In [40]:
import env
import pandas as pd
import pyspark

from pyspark.sql.functions import concat, sum, avg, min, max, count, mean, lit, when, asc, desc, expr

spark = pyspark.sql.SparkSession.builder.getOrCreate()

1. Create a spark data frame that contains your favorite programming languages.
* The name of the column should be language

In [14]:
languages_pandas_df = pd.DataFrame(["java", "python", "javascript", "html", "css", "sql"])
languages_pandas_df.columns = ['language']
languages_pandas_df

,language
0,java
1,python
2,javascript
3,html
4,css
5,sql


In [16]:
langauges_df = spark.createDataFrame(languages_pandas_df)

In [16]:
langauges_df.select("*").show()

+----------+
|  language|
+----------+
|      java|
|    python|
|javascript|
|      html|
|       css|
|       sql|
+----------+



* View the schema of the dataframe

In [17]:
langauges_df.printSchema()

root
 |-- language: string (nullable = true)



* Output the shape of the dataframe

In [13]:
langauges_df.select(langauges_df.language).describe().show()

+-------+--------+
|summary|language|
+-------+--------+
|  count|       6|
|   mean|    null|
| stddev|    null|
|    min|     css|
|    max|     sql|
+-------+--------+



* Show the first 5 records in the dataframe

In [19]:
langauges_df.select("*").show(5)

+----------+
|  language|
+----------+
|      java|
|    python|
|javascript|
|      html|
|       css|
+----------+
only showing top 5 rows



2. Load the mpg dataset as a spark dataframe.

In [20]:
from pydataset import data

In [21]:
mpg_pandas_df = data('mpg')

In [22]:
mpg_df = spark.createDataFrame(mpg_pandas_df)
mpg_df.select("*").show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



* Create 1 column of output that contains a message for each vehicle like the one below:
> `The 1999 audi a4 has a 4 cylinder engine.`

In [39]:
mpg_df = mpg_df.select( "*",
    concat(
        lit("The "), mpg_df.year, lit(" "), mpg_df.manufacturer, lit(" "), mpg_df.model, lit(" has a "), mpg_df.cyl, lit(" cylinder engine.")
    ).alias("engine_description")
)

mpg_df.show(5, truncate=False)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+-----------------------------------------+
|manufacturer|model|displ|year|cyl|trans     |drv|cty|hwy|fl |class  |engine_description                       |
+------------+-----+-----+----+---+----------+---+---+---+---+-------+-----------------------------------------+
|audi        |a4   |1.8  |1999|4  |auto(l5)  |f  |18 |29 |p  |compact|The 1999 audi a4 has a 4 cylinder engine.|
|audi        |a4   |1.8  |1999|4  |manual(m5)|f  |21 |29 |p  |compact|The 1999 audi a4 has a 4 cylinder engine.|
|audi        |a4   |2.0  |2008|4  |manual(m6)|f  |20 |31 |p  |compact|The 2008 audi a4 has a 4 cylinder engine.|
|audi        |a4   |2.0  |2008|4  |auto(av)  |f  |21 |30 |p  |compact|The 2008 audi a4 has a 4 cylinder engine.|
|audi        |a4   |2.8  |1999|6  |auto(l5)  |f  |16 |26 |p  |compact|The 1999 audi a4 has a 6 cylinder engine.|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+--------------------------

* Transform the trans column so that it only contains either manual or auto.

In [50]:
mpg_df.select(when(expr("trans LIKE 'auto%'"), "auto").otherwise("manual").alias("trans")).show()

+------+
| trans|
+------+
|  auto|
|manual|
|manual|
|  auto|
|  auto|
|manual|
|  auto|
|manual|
|  auto|
|manual|
|  auto|
|  auto|
|manual|
|  auto|
|manual|
|  auto|
|  auto|
|  auto|
|  auto|
|  auto|
+------+
only showing top 20 rows

